In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
import geopandas as gpd
import pandas as pd
import math
import calendar

In [2]:
#Read in LA neighborhood map
#Retrieved from http://boundaries.latimes.com/set/la-county-neighborhoods-current/
nbd= gpd.read_file("data/la-county-neighborhoods-current_nocatalina.geojson")
#nbd['name'][nbd.name.str.contains('Avalon')] 

In [4]:
nbd.head()

,kind,external_id,name,slug,set,metadata,resource_uri,geometry
0,L.A. County Neighborhood (Current),acton,Acton,acton-la-county-neighborhood-current,/1.0/boundary-set/la-county-neighborhoods-curr...,"{'sqmi': 39.3391089485, 'type': 'unincorporate...",/1.0/boundary/acton-la-county-neighborhood-cur...,"MULTIPOLYGON (((-118.20262 34.53899, -118.1894..."
1,L.A. County Neighborhood (Current),adams-normandie,Adams-Normandie,adams-normandie-la-county-neighborhood-current,/1.0/boundary-set/la-county-neighborhoods-curr...,"{'sqmi': 0.805350187789, 'type': 'segment-of-a...",/1.0/boundary/adams-normandie-la-county-neighb...,"MULTIPOLYGON (((-118.30901 34.03741, -118.3004..."
2,L.A. County Neighborhood (Current),agoura-hills,Agoura Hills,agoura-hills-la-county-neighborhood-current,/1.0/boundary-set/la-county-neighborhoods-curr...,"{'sqmi': 8.14676029818, 'type': 'standalone-ci...",/1.0/boundary/agoura-hills-la-county-neighborh...,"MULTIPOLYGON (((-118.76193 34.16820, -118.7263..."
3,L.A. County Neighborhood (Current),agua-dulce,Agua Dulce,agua-dulce-la-county-neighborhood-current,/1.0/boundary-set/la-county-neighborhoods-curr...,"{'sqmi': 31.4626319451, 'type': 'unincorporate...",/1.0/boundary/agua-dulce-la-county-neighborhoo...,"MULTIPOLYGON (((-118.25468 34.55830, -118.2555..."
4,L.A. County Neighborhood (Current),alhambra,Alhambra,alhambra-la-county-neighborhood-current,/1.0/boundary-set/la-county-neighborhoods-curr...,"{'sqmi': 7.62381430605, 'type': 'standalone-ci...",/1.0/boundary/alhambra-la-county-neighborhood-...,"MULTIPOLYGON (((-118.12175 34.10504, -118.1168..."


In [5]:
#Get boundary file to align with County Public Health nomenclature
#Still more need to be understood and fixed
nbd.to_csv("nocatalina.csv", columns=['name'])
nbd.replace({'Bel-Air': 'Bel Air', 
             'West Whittier-Los Nietos':'West Whittier/Los Nietos',
             'Unincorporated Santa Monica Mountains':'Santa Monica Mountains',
            'Mount Washington':'Mt. Washington',
            'Lopez/Kagel Canyons':'Kagel/Lopez Canyons',
            'View Park-Windsor Hills':'View Park/Windsor Hills',
            'Lake View Terrace':'Lakeview Terrace',
            'Playa del Rey':'Playa Del Rey', #should really be fixed in source
            'Silver Lake': 'Silverlake',#should really be fixed in source
            'Mid-City': 'Mid-city',#should really be fixed in source
            'East San Gabriel': 'Northeast San Gabriel' #correct?
#            'Unincorporated Catalina Island': 'Santa Catalina Island'
            }, inplace=True)

In [6]:
##GEO LABELING setup
#Read in LA county regions - for use in labeling
regs = gpd.read_file("data/la-county-regions-current.geojson")
#regs
regsd = regs.drop([0, 5, 6, 7, 10,14]) #Keep only desired regions

#Rename Central LA Hollywood to better match its centroid
d = {'Central L.A.':'Hollywood'}
regsd = regsd.replace(d)
regsd

#Store Long Beach centroid to use for plotting 'Harbor' data label - so that it's not in the ocean, due to Catalina
nbd[nbd['name'].str.contains('Long Beach')].geometry.centroid
longbeach = (-118.16071, 33.80522)

In [13]:
def daily_map(mm, dd, usemax=0, labels=False, savename=False):
    
    month = str(mm).zfill(2)
    day = str(dd).zfill(2)
    strmonth = calendar.month_abbr[mm]

    #Read in daily update
    #Retrieved from http://publichealth.lacounty.gov/media/Coronavirus/locations.htm
    dailyrepi = pd.read_csv('data/covid_'+month+day+'.csv')

    #We will drop suppressed case counts
    dailyrepi['count'] = dailyrepi['count'].apply(pd.to_numeric, errors='coerce')

    dailyrep = dailyrepi[dailyrepi['count'].notna()]

    #Find values of note: number under investigation, total
    underinv = dailyrep[dailyrep['city'].str.contains('Under Investigation')]['count'].item()
    nconfirmed = dailyrep[dailyrep['city'].str.contains('Total')]['count'].item()
    
    #Use a persistent scalebar across plots
    if usemax:
        vmax = usemax
    else:
        vmax = dailyrep[~dailyrep["city"].str.contains("Under Investigation") & ~dailyrep["city"].str.contains("Total") & ~dailyrep["city"].str.contains("AGGREGATE") ]['count'].max()
        print('Normalizing to', vmax)#, '\n For context:', ~dailyrep["count"].str.contains("suppressed").nlargest(5))
        
    #Add geographic information for neighborhoods
    merged = nbd.set_index('name').join(dailyrep.set_index('city'))
    


    #Plot
    fig, ax = plt.subplots(1,1, figsize=(10, 11))
    nbd.plot(ax=ax, color='#eeeeee',edgecolor='white', linewidth=1)
    divider = make_axes_locatable(ax)
    cax1 = divider.append_axes("right", size="5%", pad=0.05) #cax1 = divider.append_axes("bottom", size="5%", pad=-3)
    ax = merged.plot(column='count', vmin =0, vmax = vmax, cmap='Reds', ax=ax, cax=cax1, legend=True,edgecolor='white', linewidth=0.4)#legend_kwds={"aspect":"10"})#orientation": "horizontal"})
    
    #Add labels for case counts
    # merged.apply(lambda x: ax.annotate(x.count(), xy=x.geometry.centroid.coords[0], ha='center'),axis=1);    
    for idx, row in merged.iterrows():
        if not math.isnan(row['count']):
            if (row.external_id=='long-beach'):
                ha = 'left'
                ax.annotate(s='  '+str(int(row['count'])), xy=row.geometry.centroid.coords[0], horizontalalignment=ha,color=color, fontsize=6, weight='bold')
            else:
                ha = 'center'
                color='white'
                ax.annotate(s=int(row['count']), xy=row.geometry.centroid.coords[0], horizontalalignment=ha, color=color, fontsize=6, weight='bold')

    #Add labels for regions
    # regsd.plot(facecolor='b',edgecolor='k', linewidth=1, ax=ax, alpha = 0.2) #Plot region boundaries
    if labels:
        for idx, row in regsd.iterrows():
            if (idx==2):
                fs = 8
                ax.annotate(s=row['name'], xy=row.geometry.centroid.coords[0], horizontalalignment='center', color='#767676', fontsize=fs, alpha = 0.95, weight='bold')
            elif (idx==4):
                fs=11
                ax.annotate(s=row['name'], xy=longbeach, horizontalalignment='right', verticalalignment='top', color='#767676', fontsize=fs, alpha = 0.95, weight='bold')
            else:
                fs = 11
                ax.annotate(s=row['name'], xy=row.geometry.centroid.coords[0], horizontalalignment='center', color='#767676', fontsize=fs, alpha = 0.95, weight='bold')

    #Adjust plot display
    ax.set_ylim([33.7, 34.85])
    ax.axis('off')
    ax.set_title('Confirmed COVID-19 Cases in LA County', fontdict={'fontsize': '20', 'fontweight' : 'demibold'})
    ax.annotate(strmonth+' '+str(dd),xy=(0.1, .2),  xycoords='figure fraction', horizontalalignment='left', verticalalignment='top', fontsize=20, color='k', weight='demibold')
    ax.annotate('Total: '+str(int(nconfirmed)),xy=(0.1, .15),  xycoords='figure fraction', horizontalalignment='left', verticalalignment='top', fontsize=18, color='#555555', weight='demibold')
    ax.annotate('Under investigation: '+str(int(underinv))+' (not shown)',xy=(0.1, .11),  xycoords='figure fraction', horizontalalignment='left', verticalalignment='top', fontsize=16, color='#555555')#, weight='demibold')
    ax.annotate('Source: Los Angeles County Department of Public Health',xy=(0.1, .05),  xycoords='figure fraction', horizontalalignment='left', verticalalignment='top', fontsize=12, color='#666666')
    if savename:
        fout = 'maps/covid_la_'+month+day+savename+'.png' 
    else:
        fout = 'maps/covid_la_'+month+day+'.png'
    fig.savefig(fout)#, bbox_inches = 'tight')
    fig.clf()
    return vmax

In [14]:
#Create plots for the specified days
#month = 3
#days = [20,23, 24, 25,26, 27,28, 29, 30,31]
dates = [320, 323, 324, 325, 326, 327,328, 329, 330, 331, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411] #331, 

import pdb

vmax = 0
for idx,date in enumerate(reversed(dates)):
    month = int(date/100)
    day = date-month*100
    print('MO', month, 'DA', day)
    if (idx==0):
        vmax = daily_map(month,day, usemax=False, savename='_0',labels=False)
        vmax = daily_map(month,day, usemax=False, savename='_1', labels=True)
    else:
        if (idx==len(dates)-1):
            daily_map(month,day, usemax=vmax, savename='_1', labels=False)
            daily_map(month,day, usemax=vmax, savename='_0', labels=True)
        else:
            daily_map(month,day, usemax=vmax)

MO 4 DA 11
Normalizing to 316.0
Normalizing to 316.0
MO 4 DA 10
MO 4 DA 9
MO 4 DA 8
MO 4 DA 7
MO 4 DA 6
MO 4 DA 5
MO 4 DA 4
MO 4 DA 3
MO 4 DA 2
MO 4 DA 1
MO 3 DA 31
MO 3 DA 30
MO 3 DA 29
MO 3 DA 28
MO 3 DA 27
MO 3 DA 26
MO 3 DA 25
MO 3 DA 24
MO 3 DA 23


<ipython-input-13-0951a80d9f15>:33: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(1,1, figsize=(10, 11))


MO 3 DA 20


<ipython-input-13-0951a80d9f15>:33: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(1,1, figsize=(10, 11))
<ipython-input-13-0951a80d9f15>:33: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(1,1, figsize=(10, 11))


<Figure size 720x792 with 0 Axes>

...

<Figure size 720x792 with 0 Axes>

<Figure size 720x792 with 0 Axes>

<Figure size 720x792 with 0 Axes>

<Figure size 720x792 with 0 Axes>

<Figure size 720x792 with 0 Axes>

To create an animated gif, use imagemagick locally.  Currently using:

convert -delay 90 -loop 0 *png covid_la.gif


In [ ]:
#Write out files
import glob
import os

#path = 'data'
#all_files = glob.glob(path + "/*.csv")

df = pd.read_csv('data/covid_0320.csv').set_index('city')

df = df.fillna('null').rename(columns={'count': '03/20/2020'})

In [ ]:

dates = [323, 324, 325, 326, 327,328, 329, 330, 331, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411] #331, 
for idx, date in enumerate(dates):
    strdate = str(date).zfill(4)
    month = int(date/100)
    day = str(date-month*100)
    filename = 'data/covid_'+strdate+'.csv'
    if (filename == 'data/covid_0322_s.csv'):
        continue
    print('Working on ',filename)
    
    df0 = pd.read_csv(filename).set_index('city') 

    df0 = df0.rename(columns={'count' : str(month)+'/'+day+'/2020'})
    df0 = df0.rename(columns={'rate' : 'Rate'+str(month)+'/'+day+'/2020'})
#    df0=df0.loc[df0.index.drop_duplicates()]
    df = pd.concat([df, df0], join='outer', axis=1)
    print(df.head())

print('Once')
#Check totals
print(df[df.index.isin(['Total'])])
print('Twice')
#Write out file

In [ ]:
#Write out files for Flourish plots

df.drop(['Total', 'Los Angeles - AGGREGATE']).to_csv(
#df[~df["index"].str.contains("Under Investigation") & ~df["index"].str.contains("Total") & ~df["index"].str.contains("AGGREGATE")].to_csv(
    "./data/combined_reports_"+strdate+"_nototal.csv",
    index=True,
    encoding="utf-8"
)



#df.fillna(0).T.to_csv(
#    "./data/combined_reports_"+strdate+".csv",
#    index=True,
#    encoding="utf-8"
#)


mymerged = nbd.set_index('name').join(df)
mymerged.to_file(
    "./data/combined_reports_regs_"+strdate+".json",
    index=True, driver="GeoJSON"
)

In [ ]:
#Check merged output
pd.set_option('display.max_rows', None)
mymerged

In [ ]:
#Separate test
df = pd.read_csv('data/covid_0330.csv')
merged = nbd.merge(df, left_on="name", right_on="city", indicator=True)
nbd.set_index('name')
df.set_index('city')
#df['city'].equals(nbd['name'])
pd.concat([nbd, df]).drop_duplicates(keep=False)

In [ ]:
#Test out plotting a barchart
fig, ax = plt.subplots(1,1, figsize=(12, 20))
df = pd.read_csv('data/covid_0324.csv')
df.set_index('city', inplace=True, drop=True)
df.drop(['Under Investigation']).sort_values(by=['count'],ascending=1).plot(kind='barh', ax=ax, legend=False)
df['count'].max()